!pip install psycopg2 - installs psycopg2 python package . It allows python programs to connect and interact with postgreSQL databases. WHen using SQLAlchemy, it is required as a PostgreSQL driver
!pip install python-dotenv - installs python-dotenv package which is used to load envronmental variables from .env file into Python

In [2]:
# Import dependencies
import pandas as pd   #for data manipulation and analysis
import numpy as np    #for numerical computations.
pd.set_option('max_colwidth', 400)         #Sets the max column width for Pandas output.
from sqlalchemy import create_engine , MetaData, Table, Column, Integer, String, Float, TIMESTAMP,ForeignKey    #Imports SQLAlchemy components for database interactions and table definitions.
from dotenv import load_dotenv      # For loading environment variables from a .env file.
import os                                #Provides functions to interact with the operating system.
import psycopg2                         #PostgreSQL adapter for Python to connect and execute SQL commands
from sqlalchemy.ext.declarative import declarative_base    #Defines a base class for SQLAlchemy ORM models.
from sqlalchemy.orm import relationship, sessionmaker     #Manages relationships between tables and handles database sessions.

# Define Base for ORM models
Base = declarative_base()

C:\Users\rince\AppData\Local\Temp\ipykernel_18828\1082748664.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# For the security of the credentals like username and password, we save it in .env file . .env file keep sensitive information separate from the main code
# loads environment variables from a .env file into Python. 
load_dotenv()

#  Get PostgreSQL username, password from environment variables
username = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")

# Set the database name.Here we dont directly create crowdfunding database, because if there is any preexisting crowdfunding database, we cannot drop that once we are in that database.
database = "postgres"

# Define the database host.This implies that the database is running locally.
host = "localhost"

# Set the default PostgreSQL port
port = "5432"

# Construct the PostgreSQL connection URL
url = f"postgresql://{username}:{password}@{host}:{port}/{database}"

# Create a SQLAlchemy engine for database connection, `echo=True` allows printing all SQL queries executed in the terminal or console
engine = create_engine(url, echo=True)

# Connect using psycopg2
# Establish a connection to PostgreSQL using psycopg2
conn = psycopg2.connect(url)

#PostgreSQL transactions require an explicit COMMIT or ROLLBACK before changes are finalized.
# Setting conn.autocommit = True disables this behavior, meaning every SQL command executes immediately without needing an explicit commit.
conn.autocommit = True  # Enable auto-commit (important for creating a DB)

# Create a cursor object to interact with the database
cursor = conn.cursor()


In [4]:
# Define the database name  
database_name = "crowdfunding_db"

# SQL query to drop the database if it already exists  
drop_query = f"DROP DATABASE IF EXISTS {database_name};"

# SQL query to create a new database  
create_query = f"CREATE DATABASE {database_name};"

try:
    # Execute the SQL query to drop the database (if it exists)
    cursor.execute(drop_query)
    print(f"Database '{database_name}' dropped successfully!")
    
    # Execute the SQL query to create a new database 
    cursor.execute(create_query)
    print(f"Database '{database_name}' created successfully!")

# Print any error that occurs during execution  
except Exception as e:
    print("Error:", e)



# Close the connection to the database 
conn.close()

# Close the connection
cursor.close()

Database 'crowdfunding_db' dropped successfully!
Database 'crowdfunding_db' created successfully!


In [5]:

# Establish a new connection to the PostgreSQL database named "crowdfunding_db"
conn = psycopg2.connect(
    dbname="crowdfunding_db",    # Specify the database name
    user=os.getenv("PG_USER"),   # Fetch the PostgreSQL username and password from environment variables
    password=os.getenv("PG_PASS"),
    host="localhost",
    port="5432"
)

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Execute a SQL query to get the name of the currently connected database
cursor.execute("SELECT current_database();")
print("Connected to:", cursor.fetchone()[0])
print(f"Connected successfully!")



Connected to: crowdfunding_db
Connected successfully!


### Extract the crowdfunding.xlsx Data

In [6]:
# Read the data into a Pandas DataFrame
crowdfunding_info_df = pd.read_excel('Resources/crowdfunding.xlsx')
crowdfunding_info_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays


In [7]:
# Get a brief summary of the crowdfunding_info DataFrame.
crowdfunding_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   cf_id                    1000 non-null   int64 
 1   contact_id               1000 non-null   int64 
 2   company_name             1000 non-null   object
 3   blurb                    1000 non-null   object
 4   goal                     1000 non-null   int64 
 5   pledged                  1000 non-null   int64 
 6   outcome                  1000 non-null   object
 7   backers_count            1000 non-null   int64 
 8   country                  1000 non-null   object
 9   currency                 1000 non-null   object
 10  launched_at              1000 non-null   int64 
 11  deadline                 1000 non-null   int64 
 12  staff_pick               1000 non-null   bool  
 13  spotlight                1000 non-null   bool  
 14  category & sub-category  1000 non-null   

### Create the Category and Subcategory DataFrames
---
**Create a Category DataFrame that has the following columns:**
- A "category_id" column that is numbered sequential form 1 to the length of the number of unique categories.
- A "category" column that has only the categories.

Export the DataFrame as a `category.csv` CSV file.

**Create a SubCategory DataFrame that has the following columns:**
- A "subcategory_id" column that is numbered sequential form 1 to the length of the number of unique subcategories.
- A "subcategory" column that has only the subcategories. 

Export the DataFrame as a `subcategory.csv` CSV file.

In [8]:
# Get the crowdfunding_info_df columns.
crowdfunding_info_df.columns

Index(['cf_id', 'contact_id', 'company_name', 'blurb', 'goal', 'pledged',
       'outcome', 'backers_count', 'country', 'currency', 'launched_at',
       'deadline', 'staff_pick', 'spotlight', 'category & sub-category'],
      dtype='object')

In [9]:
# Assign the category and subcategory values to category and subcategory columns.
# Using str.split() function, where '/' is the delimiter. expand = True splits category & sub-category into two separate columns.

crowdfunding_info_df[['category','subcategory']] = crowdfunding_info_df['category & sub-category'].str.split('/', expand=True)
crowdfunding_info_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


In [10]:
# Get the unique categories and subcategories in separate lists.
# Extracts category column from crowdfunding_info_df. 
#.unique() removes any duplicate values in the column
#.tolist() converts the array into python list.

categories =  crowdfunding_info_df['category'].unique().tolist()

subcategories = crowdfunding_info_df['subcategory'].unique().tolist()

print(categories)
print(subcategories)

['food', 'music', 'technology', 'theater', 'film & video', 'publishing', 'games', 'photography', 'journalism']
['food trucks', 'rock', 'web', 'plays', 'documentary', 'electric music', 'drama', 'indie rock', 'wearables', 'nonfiction', 'animation', 'video games', 'shorts', 'fiction', 'photography books', 'radio & podcasts', 'metal', 'jazz', 'translations', 'television', 'mobile games', 'world music', 'science fiction', 'audio']


In [11]:
# Get the number of distinct values in the categories and subcategories lists.
#printing the length of distinct values in categories and subcategories.
 
print(len(categories))
print(len(subcategories))

9
24


In [12]:
# Create numpy arrays from 1-9 for the categories and 1-24 for the subcategories.
category_ids = np.arange(1, 10)
subcategory_ids = np.arange(1, 25)

print(category_ids)
print(subcategory_ids)

[1 2 3 4 5 6 7 8 9]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24]


In [13]:
# Use a list comprehension to add "cat" to each category_id. 
# For each cat_id in category_ids, the string "cat" is prefixed, and the result is added to the list.

cat_ids = [f"cat{cat_id}" for cat_id in category_ids]

# Use a list comprehension to add "subcat" to each subcategory_id.    
# For each sub_id in subcategory_ids, the string "subcat" is prefixed, and the result is added to the list.

scat_ids = [f"subcat{sub_id}" for sub_id in subcategory_ids]
    
print(cat_ids)
print(scat_ids)

['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
['subcat1', 'subcat2', 'subcat3', 'subcat4', 'subcat5', 'subcat6', 'subcat7', 'subcat8', 'subcat9', 'subcat10', 'subcat11', 'subcat12', 'subcat13', 'subcat14', 'subcat15', 'subcat16', 'subcat17', 'subcat18', 'subcat19', 'subcat20', 'subcat21', 'subcat22', 'subcat23', 'subcat24']


In [14]:
# Create a category DataFrame with the category_id array as the category_id and categories list as the category name.
#Create a Pandas DataFrame from a dictionary. The DataFrame has two columns category_id and category. Each row corresponds to a pair of values from cat_ids and categories.

category_df = pd.DataFrame({"category_id": cat_ids,
                            "category": categories})

# Create a category DataFrame with the subcategory_id array as the subcategory_id and subcategories list as the subcategory name. 
#Create a Pandas DataFrame from a dictionary. The DataFrame has two columns subcategory_id and subcategory.
#  Each row corresponds to a pair of values from scat_ids and subcategories.

subcategory_df = pd.DataFrame({"subcategory_id": scat_ids,
                            "subcategory": subcategories})

In [15]:
category_df

,category_id,category
0,cat1,food
1,cat2,music
2,cat3,technology
3,cat4,theater
4,cat5,film & video
5,cat6,publishing
6,cat7,games
7,cat8,photography
8,cat9,journalism


In [16]:
subcategory_df

,subcategory_id,subcategory
0,subcat1,food trucks
1,subcat2,rock
2,subcat3,web
3,subcat4,plays
4,subcat5,documentary
5,subcat6,electric music
6,subcat7,drama
7,subcat8,indie rock
8,subcat9,wearables
9,subcat10,nonfiction


In [17]:
# Export categories_df and subcategories_df as CSV files.
category_df.to_csv("Resources/category.csv", index=False)

subcategory_df.to_csv("Resources/subcategory.csv", index=False)

### Campaign DataFrame
----
**Create a Campaign DataFrame that has the following columns:**
- The "cf_id" column.
- The "contact_id" column.
- The “company_name” column.
- The "blurb" column is renamed as "description."
- The "goal" column.
- The "goal" column is converted to a `float` datatype.
- The "pledged" column is converted to a `float` datatype. 
- The "backers_count" column. 
- The "country" column.
- The "currency" column.
- The "launched_at" column is renamed as "launch_date" and converted to a datetime format. 
- The "deadline" column is renamed as "end_date" and converted to a datetime format.
- The "category_id" with the unique number matching the “category_id” from the category DataFrame. 
- The "subcategory_id" with the unique number matching the “subcategory_id” from the subcategory DataFrame.
- And, create a column that contains the unique four-digit contact ID number from the `contact.xlsx` file.
 

Then export the DataFrame as a `campaign.csv` CSV file.


In [18]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df. 
campaign_df = crowdfunding_info_df.copy()
campaign_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


In [19]:
# Rename the blurb, launched_at, and deadline columns.
campaign_df = campaign_df.rename(columns={'blurb':'description', 'launched_at':'launched_date', 'deadline':'end_date'})
campaign_df.head()


,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


In [20]:
# Convert the goal and pledged columns to a `float` data type.
campaign_df[['goal', 'pledged']] = campaign_df[['goal', 'pledged']].astype(float)
campaign_df.head()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


In [21]:
# Check the datatypes
campaign_df.dtypes

cf_id                        int64
contact_id                   int64
company_name                object
description                 object
goal                       float64
pledged                    float64
outcome                     object
backers_count                int64
country                     object
currency                    object
launched_date                int64
end_date                     int64
staff_pick                    bool
spotlight                     bool
category & sub-category     object
category                    object
subcategory                 object
dtype: object

In [22]:
# Format the launched_date and end_date columns to datetime format
from datetime import datetime as dt
campaign_df['launched_date'] = pd.to_datetime(campaign_df['launched_date'], unit='s', errors='coerce')
campaign_df['end_date'] = pd.to_datetime(campaign_df['end_date'], unit='s', errors='coerce')
campaign_df.head()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13 06:00:00,2021-03-01 06:00:00,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25 06:00:00,2021-05-25 05:00:00,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,2020-12-17 06:00:00,2021-12-30 06:00:00,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,2021-10-21 05:00:00,2022-01-17 06:00:00,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,2020-12-21 06:00:00,2021-08-23 05:00:00,False,False,theater/plays,theater,plays


In [23]:
# Merge the campaign_df with the category_df on the "category" column and 
# the subcategory_df on the "subcategory" column.
campaign_category_df = pd.merge(campaign_df, category_df, on='category')
campaign_merged_df = pd.merge(campaign_category_df, subcategory_df, on='subcategory')
campaign_merged_df.tail(10)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory,category_id,subcategory_id
990,1561,5500,Ortiz-Roberts,Devolved foreground customer loyalty,7800.0,6839.0,failed,64,US,USD,2021-06-09 05:00:00,2021-06-18 05:00:00,False,True,film & video/drama,film & video,drama,cat5,subcat7
991,2632,3494,Ramirez LLC,Reduced reciprocal focus group,9800.0,11091.0,successful,241,US,USD,2020-12-09 06:00:00,2021-05-26 05:00:00,False,True,music/rock,music,rock,cat2,subcat2
992,439,3924,Morrow Inc,Networked global migration,3100.0,13223.0,successful,132,US,USD,2020-06-14 05:00:00,2021-02-09 06:00:00,False,True,film & video/drama,film & video,drama,cat5,subcat7
993,461,3521,Erickson-Rogers,De-engineered even-keeled definition,9800.0,7608.0,canceled,75,IT,EUR,2021-07-03 05:00:00,2021-07-08 05:00:00,False,True,photography/photography books,photography,photography books,cat8,subcat15
994,1419,5262,"Leach, Rich and Price",Implemented bi-directional flexibility,141100.0,74073.0,failed,842,US,USD,2021-11-15 06:00:00,2021-12-07 06:00:00,False,True,publishing/translations,publishing,translations,cat6,subcat19
995,2986,3684,Manning-Hamilton,Vision-oriented scalable definition,97300.0,153216.0,successful,2043,US,USD,2020-12-29 06:00:00,2021-05-30 05:00:00,False,True,food/food trucks,food,food trucks,cat1,subcat1
996,2031,5784,Butler LLC,Future-proofed upward-trending migration,6600.0,4814.0,failed,112,US,USD,2021-10-15 05:00:00,2021-11-30 06:00:00,False,False,theater/plays,theater,plays,cat4,subcat4
997,1627,1498,Ball LLC,Right-sized full-range throughput,7600.0,4603.0,canceled,139,IT,EUR,2021-11-06 05:00:00,2021-12-10 06:00:00,False,False,theater/plays,theater,plays,cat4,subcat4
998,2175,6073,"Taylor, Santiago and Flores",Polarized composite customer loyalty,66600.0,37823.0,failed,374,US,USD,2020-10-08 05:00:00,2021-04-11 05:00:00,False,True,music/indie rock,music,indie rock,cat2,subcat8
999,1788,4939,"Hernandez, Norton and Kelley",Expanded eco-centric policy,111100.0,62819.0,canceled,1122,US,USD,2020-12-30 06:00:00,2021-08-18 05:00:00,False,False,food/food trucks,food,food trucks,cat1,subcat1


In [24]:
# Drop unwanted columns
campaign_cleaned = campaign_merged_df.drop(columns=['staff_pick', 'spotlight', 'category & sub-category', 'category', 'subcategory' ])  # Flat list
campaign_cleaned.head()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100.0,0.0,failed,0,CA,CAD,2020-02-13 06:00:00,2021-03-01 06:00:00,cat1,subcat1
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400.0,14560.0,successful,158,US,USD,2021-01-25 06:00:00,2021-05-25 05:00:00,cat2,subcat2
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400.0,142523.0,successful,1425,AU,AUD,2020-12-17 06:00:00,2021-12-30 06:00:00,cat3,subcat3
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200.0,2477.0,failed,24,US,USD,2021-10-21 05:00:00,2022-01-17 06:00:00,cat2,subcat2
4,1365,2199,Larson-Little,Proactive foreground core,7600.0,5265.0,failed,53,US,USD,2020-12-21 06:00:00,2021-08-23 05:00:00,cat4,subcat4


In [25]:
# Export the DataFrame as a CSV file. 
campaign_cleaned.to_csv("Resources/campaign.csv", index=False)

### Extract the contacts.xlsx Data.

In [26]:
# Read the data into a Pandas DataFrame. Use the `header=2` parameter when reading in the data.
contact_info_df = pd.read_excel('Resources/contacts.xlsx', header=3)
contact_info_df.head()

,contact_info
0,"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}"
1,"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}"
2,"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}"
3,"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}"
4,"{""contact_id"": 2199, ""name"": ""Samuel Sorgatz"", ""email"": ""samuel.sorgatz@gmail.com""}"


### Create the Contacts DataFrame 
---
**Create a Contacts DataFrame that has the following columns:**
- A column named "contact_id"  that contains the unique number of the contact person.
- A column named "first_name" that contains the first name of the contact person.
- A column named "last_name" that contains the first name of the contact person.
- A column named "email" that contains the email address of the contact person

Then export the DataFrame as a `contacts.csv` CSV file.

### Option 1: Use Pandas to create the contacts DataFrame.

In [27]:
# Iterate through the contact_info_df and convert each row to a dictionary.
import json
dict_values = []
for index, row in contact_info_df.iterrows():
    row_filtered = list(row.values)
    dict_values.append(row_filtered)

# Print out the list of values for each row.
print(dict_values)



[['{"contact_id": 4661, "name": "Cecilia Velasco", "email": "cecilia.velasco@rodrigues.fr"}'], ['{"contact_id": 3765, "name": "Mariana Ellis", "email": "mariana.ellis@rossi.org"}'], ['{"contact_id": 4187, "name": "Sofie Woods", "email": "sofie.woods@riviere.com"}'], ['{"contact_id": 4941, "name": "Jeanette Iannotti", "email": "jeanette.iannotti@yahoo.com"}'], ['{"contact_id": 2199, "name": "Samuel Sorgatz", "email": "samuel.sorgatz@gmail.com"}'], ['{"contact_id": 5650, "name": "Socorro Luna", "email": "socorro.luna@hotmail.com"}'], ['{"contact_id": 5889, "name": "Carolina Murray", "email": "carolina.murray@knight.com"}'], ['{"contact_id": 4842, "name": "Kayla Moon", "email": "kayla.moon@yahoo.de"}'], ['{"contact_id": 3280, "name": "Ariadna Geisel", "email": "ariadna.geisel@rangel.com"}'], ['{"contact_id": 5468, "name": "Danielle Ladeck", "email": "danielle.ladeck@scalfaro.net"}'], ['{"contact_id": 3064, "name": "Tatiana Thompson", "email": "tatiana.thompson@hunt.net"}'], ['{"contact_id

In [28]:
import json

# Parse the JSON strings into dictionaries
parsed_dict_values = [json.loads(value[0]) for value in dict_values]

# Create a contact_info DataFrame and add each list of values, i.e., each row to the 'contact_id', 'name', 'email' columns.
contact_info_df = pd.DataFrame(parsed_dict_values, columns=['contact_id', 'name', 'email'])
contact_info_df.head()



,contact_id,name,email
0,4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana Ellis,mariana.ellis@rossi.org
2,4187,Sofie Woods,sofie.woods@riviere.com
3,4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel Sorgatz,samuel.sorgatz@gmail.com


In [29]:
# Check the datatypes.
contact_info_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   name        1000 non-null   object
 2   email       1000 non-null   object
dtypes: int64(1), object(2)
memory usage: 23.6+ KB


In [30]:
# Create a "first_name" and "last_name" column with the first and last names from the "name" column. 
# Use the str.split() function with the space character as the delimiter.
contact_info_df[['first_name', 'last_name']] = contact_info_df['name'].str.split(' ', expand=True)
contact_info_df.head(10)

,contact_id,name,email,first_name,last_name
0,4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr,Cecilia,Velasco
1,3765,Mariana Ellis,mariana.ellis@rossi.org,Mariana,Ellis
2,4187,Sofie Woods,sofie.woods@riviere.com,Sofie,Woods
3,4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com,Jeanette,Iannotti
4,2199,Samuel Sorgatz,samuel.sorgatz@gmail.com,Samuel,Sorgatz
5,5650,Socorro Luna,socorro.luna@hotmail.com,Socorro,Luna
6,5889,Carolina Murray,carolina.murray@knight.com,Carolina,Murray
7,4842,Kayla Moon,kayla.moon@yahoo.de,Kayla,Moon
8,3280,Ariadna Geisel,ariadna.geisel@rangel.com,Ariadna,Geisel
9,5468,Danielle Ladeck,danielle.ladeck@scalfaro.net,Danielle,Ladeck


In [31]:
# Drop the name column
contact_info_df = contact_info_df.drop(columns=['name'])
contact_info_df.head()

,contact_id,email,first_name,last_name
0,4661,cecilia.velasco@rodrigues.fr,Cecilia,Velasco
1,3765,mariana.ellis@rossi.org,Mariana,Ellis
2,4187,sofie.woods@riviere.com,Sofie,Woods
3,4941,jeanette.iannotti@yahoo.com,Jeanette,Iannotti
4,2199,samuel.sorgatz@gmail.com,Samuel,Sorgatz


In [32]:
# Reorder the columns
contact_info_df = contact_info_df[['contact_id', 'first_name', 'last_name', 'email']]
contact_info_df.head(10)

,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org
2,4187,Sofie,Woods,sofie.woods@riviere.com
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com
5,5650,Socorro,Luna,socorro.luna@hotmail.com
6,5889,Carolina,Murray,carolina.murray@knight.com
7,4842,Kayla,Moon,kayla.moon@yahoo.de
8,3280,Ariadna,Geisel,ariadna.geisel@rangel.com
9,5468,Danielle,Ladeck,danielle.ladeck@scalfaro.net


In [33]:
# Check the datatypes one more time before exporting as CSV file.
contact_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   contact_id  1000 non-null   int64 
 1   first_name  1000 non-null   object
 2   last_name   1000 non-null   object
 3   email       1000 non-null   object
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [34]:
# Export the DataFrame as a CSV file. 
contact_info_df.to_csv("Resources/contact_info.csv", index=False)


# Dataframes for which we are creating tables in postgres are category_df, subcategory_df, contact_info , campaign_cleaned, contact_info_df

In [35]:
# Define the Category table as a Python class for SQLAlchemy ORM
class Category(Base):
    __tablename__ = 'category'      # Set the table name in the PostgreSQL database
    
    category_id = Column(String, primary_key=True)          # Define 'category_id' as the primary key (string type)
    category = Column(String)                           # Define 'category' column (string type) to store category names

    # # Define a relationship with the Campaign table
    campaigns = relationship("Campaign", back_populates="category")
# This establishes a one-to-many  relationship between Category and Campaign.
# The "Campaign" class must have a foreign key referencing 'category_id' in its table.
# back_populates="category"` ensures that the relationship is bidirectional.

In [36]:
# Subcategory Table (ORM)
class Subcategory(Base):
    __tablename__ = 'subcategory'
    
    subcategory_id = Column(String, primary_key=True)
    subcategory = Column(String)

    # Relationship to Campaign
    campaigns = relationship("Campaign", back_populates="subcategory")

In [37]:
# Contacts Table (ORM)
class Contact(Base):
    __tablename__ = 'contacts'
    
    contact_id = Column(Integer, primary_key=True)
    first_name = Column(String)
    last_name = Column(String)
    email = Column(String)

    # Relationship to Campaign
    campaigns = relationship("Campaign", back_populates="contact")

In [38]:
# Campaign Table (ORM)
class Campaign(Base):
    __tablename__ = 'campaign'
    
    cf_id = Column(Integer, primary_key=True)
    contact_id = Column(Integer, ForeignKey('contacts.contact_id'))
    company_name = Column(String)
    description = Column(String)
    goal = Column(Float)
    pledged = Column(Float)
    outcome = Column(String)
    backers_count = Column(Integer)
    country = Column(String)
    currency = Column(String)
    launched_date = Column(TIMESTAMP)
    end_date = Column(TIMESTAMP)
    category_id = Column(String, ForeignKey('category.category_id'))
    subcategory_id = Column(String, ForeignKey('subcategory.subcategory_id'))

    # Relationships
    contact = relationship("Contact", back_populates="campaigns")
    category = relationship("Category", back_populates="campaigns")
    subcategory = relationship("Subcategory", back_populates="campaigns")

In [39]:
# Importing pprint for pretty printing the output
from pprint import pprint

# Create a new session by binding the sessionmaker to the engine
Session = sessionmaker(bind=engine)
session = Session()

# Create the tables in the database based on the defined Base classes
Base.metadata.create_all(engine)

# Iterate over each row in the 'category_df' DataFrame
for _, row in category_df.iterrows():

     # Create a new 'Category' object using the values from the current row of the DataFrame
    category = Category(category_id=row["category_id"], category=row["category"])

    # Add the 'category' object to the current session 
    session.add(category)

# Commit the transaction
session.commit()


2025-02-01 13:32:14,842 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-02-01 13:32:14,846 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-01 13:32:14,858 INFO sqlalchemy.engine.Engine select current_schema()
2025-02-01 13:32:14,861 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-01 13:32:14,871 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-02-01 13:32:14,875 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-02-01 13:32:14,887 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-01 13:32:14,918 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [40]:
# Fetch all records from the category table
categories = session.query(Category).all()

# Display results
for category in categories:
    print(f"Category ID: {category.category_id}, Name: {category.category}")
pprint(categories)


2025-02-01 13:32:15,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-02-01 13:32:15,506 INFO sqlalchemy.engine.Engine SELECT category.category_id AS category_category_id, category.category AS category_category 
FROM category
2025-02-01 13:32:15,511 INFO sqlalchemy.engine.Engine [generated in 0.00500s] {}
Category ID: cat1, Name: food
Category ID: cat2, Name: music
Category ID: cat3, Name: technology
Category ID: cat4, Name: theater
Category ID: cat5, Name: film & video
Category ID: cat6, Name: publishing
Category ID: cat7, Name: games
Category ID: cat8, Name: photography
Category ID: cat9, Name: journalism


In [41]:
# Iterate over each row in the DataFrame 'subcategory_df'
for _, row in subcategory_df.iterrows():

    # Create a new 'Subcategory' object using values from the DataFrame
    subcategory = Subcategory(subcategory_id=row["subcategory_id"], subcategory=row["subcategory"])

    # Add the newly created object to the SQLAlchemy session for staging
    session.add(subcategory)

# Commit the transaction
session.commit()

# Fetch all records from the 'Subcategory' table 
subcategories = session.query(Subcategory).all()

# Iterate over the fetched records and print each subcategory's details
for subcategory in subcategories:
    print(f"Subcategory ID: {subcategory.subcategory_id}, Name: {subcategory.subcategory}")
pprint(subcategories)

2025-02-01 13:32:15,622 INFO sqlalchemy.engine.Engine INSERT INTO subcategory (subcategory_id, subcategory) VALUES (%(subcategory_id__0)s, %(subcategory__0)s), (%(subcategory_id__1)s, %(subcategory__1)s), (%(subcategory_id__2)s, %(subcategory__2)s), (%(subcategory_id__3)s, %(subcategory__3)s), (%(subcat ... 817 characters truncated ... _21)s), (%(subcategory_id__22)s, %(subcategory__22)s), (%(subcategory_id__23)s, %(subcategory__23)s)
2025-02-01 13:32:15,628 INFO sqlalchemy.engine.Engine [generated in 0.00070s (insertmanyvalues) 1/1 (unordered)] {'subcategory__0': 'food trucks', 'subcategory_id__0': 'subcat1', 'subcategory__1': 'rock', 'subcategory_id__1': 'subcat2', 'subcategory__2': 'web', 'subcategory_id__2': 'subcat3', 'subcategory__3': 'plays', 'subcategory_id__3': 'subcat4', 'subcategory__4': 'documentary', 'subcategory_id__4': 'subcat5', 'subcategory__5': 'electric music', 'subcategory_id__5': 'subcat6', 'subcategory__6': 'drama', 'subcategory_id__6': 'subcat7', 'subcategory__7'

In [42]:
# Iterate over each row in the DataFrame `contact_info_df`
for _, row in contact_info_df.iterrows():
     # Create a new `Contact` object using values from the current row
    contacts = Contact(contact_id=row["contact_id"], first_name=row["first_name"], last_name=row["last_name"], email=row["email"])
     # Add the new contact object to the SQLAlchemy session
    session.add(contacts)

# Commit the transaction
session.commit()

# Fetch all records from the Contact` table and store them in a list
contacts = session.query(Contact).all()

# # Iterate over the retrieved records and print each contact's details
for contact in contacts:
    print(f"Contact ID: {contact.contact_id}, First_Name: {contact.first_name}, Last_name: {contact.last_name}, Email : {contact.email}")
    
# Pretty-print the entire list of contact objects
pprint(contacts)

2025-02-01 13:32:17,355 INFO sqlalchemy.engine.Engine INSERT INTO contacts (contact_id, first_name, last_name, email) VALUES (%(contact_id__0)s, %(first_name__0)s, %(last_name__0)s, %(email__0)s), (%(contact_id__1)s, %(first_name__1)s, %(last_name__1)s, %(email__1)s), (%(contact_id__2)s, %(first_name__2 ... 79279 characters truncated ... 8)s, %(email__998)s), (%(contact_id__999)s, %(first_name__999)s, %(last_name__999)s, %(email__999)s)
2025-02-01 13:32:17,359 INFO sqlalchemy.engine.Engine [generated in 0.01858s (insertmanyvalues) 1/1 (unordered)] {'first_name__0': 'Cecilia', 'last_name__0': 'Velasco', 'email__0': 'cecilia.velasco@rodrigues.fr', 'contact_id__0': 4661, 'first_name__1': 'Mariana', 'last_name__1': 'Ellis', 'email__1': 'mariana.ellis@rossi.org', 'contact_id__1': 3765, 'first_name__2': 'Sofie', 'last_name__2': 'Woods', 'email__2': 'sofie.woods@riviere.com', 'contact_id__2': 4187, 'first_name__3': 'Jeanette', 'last_name__3': 'Iannotti', 'email__3': 'jeanette.iannotti@yahoo.c

In [43]:
#Insert data into the table using ORM
for _, row in campaign_cleaned.iterrows():
    campaign_data = Campaign(cf_id=row["cf_id"], contact_id=row["contact_id"],company_name=row["company_name"],
                                description=row["description"],goal=row["goal"],pledged=row["pledged"],
                                outcome=row["outcome"],country=row["country"],currency=row["currency"],launched_date=row["launched_date"],end_date=row["end_date"])
    session.add(campaign_data)

# Commit the transaction
session.commit()

# Fetch all records from the campaign table
campaigns = session.query(Campaign).all()

# Display results
for campaign in campaigns:
    print(f"""cf ID: {campaign.cf_id}, Contact_id: {campaign.contact_id}, Company Name: {campaign.company_name}, Description: {campaign.description}, Goal: {campaign.goal},Pledged: {campaign.pledged}, Outcome: {campaign.outcome}
          Country: {campaign.country},Currency: {campaign.currency},Launched Date: {campaign.launched_date}, End Date: {campaign.end_date}""")
    
pprint(campaigns)

2025-02-01 13:32:19,085 INFO sqlalchemy.engine.Engine INSERT INTO campaign (cf_id, contact_id, company_name, description, goal, pledged, outcome, backers_count, country, currency, launched_date, end_date, category_id, subcategory_id) VALUES (%(cf_id__0)s, %(contact_id__0)s, %(company_name__0)s, %(descri ... 284295 characters truncated ... cy__999)s, %(launched_date__999)s, %(end_date__999)s, %(category_id__999)s, %(subcategory_id__999)s)
2025-02-01 13:32:19,089 INFO sqlalchemy.engine.Engine [generated in 0.04788s (insertmanyvalues) 1/1 (unordered)] {'backers_count__0': None, 'goal__0': 100.0, 'pledged__0': 0.0, 'contact_id__0': 4661, 'currency__0': 'CAD', 'cf_id__0': 147, 'subcategory_id__0': None, 'description__0': 'Pre-emptive tertiary standardization', 'company_name__0': 'Baldwin, Riley and Jackson', 'outcome__0': 'failed', 'end_date__0': Timestamp('2021-03-01 06:00:00'), 'launched_date__0': Timestamp('2020-02-13 06:00:00'), 'country__0': 'CA', 'category_id__0': None, 'backers_count